## Put the directory structure in Keras style

**ONLY DO THIS ONCE**

In [19]:
import sys, os
import collections
def structure_dir(dir_name):
    base_path = "/root/nbs/data/256_ObjectCategories_Stratified/"
    for i in range(1, 258):
        os.mkdir(base_path + "%s/%s" % (dir_name, i))
    for f in os.listdir(base_path + dir_name):
        if 'jpg' in f:
            d = int(f.split("_")[0])
            a = base_path + dir_name + "/" + f
            b = base_path + dir_name + "/" + str(d) + "/" + f
            os.rename(a, b)

In [20]:
structure_dir('valid')
structure_dir('train')
structure_dir('test')

## Imports

In [1]:
path = "data/256_ObjectCategories_Stratified/"

In [2]:
 from __future__ import division,print_function

import os, json
from glob import glob
import numpy as np
np.set_printoptions(precision=4, linewidth=100)
from matplotlib import pyplot as plt
from keras.utils.data_utils import get_file
import bcolz
%matplotlib inline

Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

WARNING (theano.gof.compilelock): Overriding existing lock by dead process '6185' (I am process '6774')
Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)


## Create an empty Keras model, then load weights

In [3]:
from keras import backend as K
from keras.layers import Flatten, Dense, Input, Conv2D, MaxPooling2D, Dropout, BatchNormalization
from keras.models import Model
from keras.preprocessing import image
from keras.applications.imagenet_utils import _obtain_input_shape

In [4]:
K.set_image_data_format('channels_first')

In [11]:
# TODO: subtract the mean?
batch_size = 64
input_shape = (3, 224, 224)
img_input = Input(shape=input_shape)
# Block 1
x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1')(img_input)
x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

# Block 2
x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

# Block 3
x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

# Block 4
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv3')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

# Block 5
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv1')(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv2')(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv3')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(x)

# dense
x = Flatten(name='flatten')(x)
x = Dense(4096, activation='relu', name='fc1')(x)
x = Dense(4096, activation='relu', name='fc2')(x)
x = Dense(1000, activation='softmax', name='predictions')(x)
model_full = Model(img_input, x, name='vgg16_full')

In [12]:
WEIGHTS_PATH = 'https://github.com/fchollet/deep-learning-models/releases/download/v0.1/vgg16_weights_th_dim_ordering_th_kernels.h5'
weights_path = get_file('vgg16_weights_th_dim_ordering_th_kernels.h5', WEIGHTS_PATH, cache_subdir='models')
model_full.load_weights(weights_path)

## Grab just the convolutional layers, and use them to featurize

In [13]:
last_conv_idx = [i for i,l in enumerate(model_full.layers) if type(l) is Conv2D][-1]
conv_layers = model_full.layers[:last_conv_idx + 2]  # max pooling is last layer

In [14]:
last_conv_layer = conv_layers[-1]
model = Model(img_input, last_conv_layer.get_output_at(0), name='vgg16_conv')

In [79]:
# Do not shuffle the data! You'll lose the label ordering
generator = image.ImageDataGenerator()
batches = generator.flow_from_directory(path + 'train', target_size=(224, 224), class_mode='categorical', shuffle=False, batch_size=batch_size)
val_batches = generator.flow_from_directory(path + 'valid', target_size=(224, 224), class_mode='categorical', shuffle=False, batch_size=batch_size)
test_batches = generator.flow_from_directory(path + 'test', target_size=(224, 224), class_mode='categorical', shuffle=False, batch_size=batch_size)

Found 20812 images belonging to 257 classes.
Found 3673 images belonging to 257 classes.
Found 6122 images belonging to 257 classes.


### Predict and save the featurized arrays

In [81]:
%%time
conv_feat = model.predict_generator(batches, int(batches.samples / batch_size) + 1)
conv_val_feat = model.predict_generator(val_batches, int(val_batches.samples / batch_size) + 1)
conv_test_feat = model.predict_generator(test_batches, int(test_batches.samples / batch_size) + 1)

INFO (theano.gof.compilelock): Refreshing lock /root/.theano/compiledir_Linux-4.4-amzn1.x86_64-x86_64-with-glibc2.2.5-x86_64-3.5.3-64/lock_dir/lock


CPU times: user 11min 18s, sys: 2min, total: 13min 19s
Wall time: 11min 22s


In [14]:
if not os.path.exists(path + 'results'):
    os.mkdir(path + 'results')

In [14]:
c_train = bcolz.carray(conv_feat, rootdir=path + 'results/conv_feat.dat')
c_train.flush()
c_val = bcolz.carray(conv_val_feat, rootdir=path + 'results/conv_val_feat.dat')
c_val.flush()
c_test = bcolz.carray(conv_test_feat, rootdir=path + 'results/conv_test_feat.dat')
c_test.flush()

## Read the featurized data

In [5]:
conv_feat = bcolz.open(path + 'results/conv_feat.dat')
conv_val_feat = bcolz.open(path + 'results/conv_val_feat.dat')
conv_test_feat = bcolz.open(path + 'results/conv_test_feat.dat')

In [10]:
conv_feat.shape

(20812, 512, 7, 7)

## Create the dense layers, then copy weights that were loaded above

In [8]:
feat_input = Input(shape=(512, 7, 7))
y = Flatten(name='flatten')(feat_input)
y = Dense(4096, activation='relu', name='fc1')(y)
y = Dense(4096, activation='relu', name='fc2')(y)
y = Dropout(rate=0.2)(y)
y = Dense(257, activation='softmax', name='predictions')(y)

In [9]:
model_top = Model(feat_input, y, name='vgg16_top')

In [44]:
for layer_name in ['fc1', 'fc2']:
    model_top.get_layer(layer_name).set_weights(model_full.get_layer(layer_name).get_weights())

In [45]:
for layer_name in ['fc1', 'fc2']:
    model_top.get_layer(layer_name).trainable = False

In [46]:
from keras.utils.np_utils import to_categorical
(val_classes, trn_classes, val_labels, trn_labels) = \
(val_batches.classes, batches.classes, to_categorical(val_batches.classes), to_categorical(batches.classes))

In [47]:
from keras.optimizers import Adam

In [48]:
model_top.compile(Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
model_top.fit(conv_feat, trn_labels, batch_size=batch_size, epochs=2, validation_data=(conv_val_feat, val_labels))

Train on 20812 samples, validate on 3673 samples
Epoch 1/2
20812/20812 [==============================] - 15s - loss: 3.6673 - acc: 0.3170 - val_loss: 1.7736 - val_acc: 0.5826
Epoch 2/2
20812/20812 [==============================] - 15s - loss: 1.5242 - acc: 0.6274 - val_loss: 1.3964 - val_acc: 0.6610


In [49]:
model_top.fit(conv_feat, trn_labels, batch_size=batch_size, epochs=4, validation_data=(conv_val_feat, val_labels))

Train on 20812 samples, validate on 3673 samples
Epoch 1/4
20812/20812 [==============================] - 15s - loss: 1.0815 - acc: 0.7207 - val_loss: 1.2822 - val_acc: 0.6885
Epoch 2/4
20812/20812 [==============================] - 15s - loss: 0.8206 - acc: 0.7810 - val_loss: 1.2241 - val_acc: 0.7062
Epoch 3/4
20812/20812 [==============================] - 15s - loss: 0.6480 - acc: 0.8236 - val_loss: 1.1945 - val_acc: 0.7130
Epoch 4/4
20812/20812 [==============================] - 15s - loss: 0.5129 - acc: 0.8562 - val_loss: 1.1845 - val_acc: 0.7169


In [50]:
model_top.optimizer.lr = 0.00001
model_top.fit(conv_feat, trn_labels, batch_size=batch_size, epochs=2, validation_data=(conv_val_feat, val_labels))

Train on 20812 samples, validate on 3673 samples
Epoch 1/2
20812/20812 [==============================] - 15s - loss: 0.4148 - acc: 0.8858 - val_loss: 1.1821 - val_acc: 0.7231
Epoch 2/2
20812/20812 [==============================] - 15s - loss: 0.3330 - acc: 0.9089 - val_loss: 1.1853 - val_acc: 0.7288


In [51]:
model_top.fit(conv_feat, trn_labels, batch_size=batch_size, epochs=4, validation_data=(conv_val_feat, val_labels))

Train on 20812 samples, validate on 3673 samples
Epoch 1/4
20812/20812 [==============================] - 15s - loss: 0.2730 - acc: 0.9296 - val_loss: 1.1904 - val_acc: 0.7275
Epoch 2/4
20812/20812 [==============================] - 15s - loss: 0.2253 - acc: 0.9422 - val_loss: 1.1891 - val_acc: 0.7264
Epoch 3/4
20812/20812 [==============================] - 15s - loss: 0.1848 - acc: 0.9566 - val_loss: 1.1892 - val_acc: 0.7332
Epoch 4/4
20812/20812 [==============================] - 15s - loss: 0.1574 - acc: 0.9663 - val_loss: 1.1951 - val_acc: 0.7307


In [52]:
model_top.save(path + 'results/vgg16_top')

## Train output + fc2

In [71]:
from keras import regularizers
feat_input = Input(shape=(512, 7, 7))
y = Flatten(name='flatten')(feat_input)
y = Dense(4096, activation='relu', name='fc1')(y)
# y = Dropout(rate=0.3)(y)
y = BatchNormalization()(y)
y = Dense(4096, activation='relu', name='fc2', kernel_regularizer=regularizers.l2(0.05))(y)
# y = Dropout(rate=0.3)(y)
y = BatchNormalization()(y)
y = Dense(257, activation='softmax', name='predictions', kernel_regularizer=regularizers.l2(0.05))(y)

In [72]:
model_fc2 = Model(feat_input, y, name='vgg16_fc2')

In [73]:
for layer_name in ['fc1']:
    model_fc2.get_layer(layer_name).set_weights(model_full.get_layer(layer_name).get_weights())

In [74]:
for layer_name in ['fc1']:
    model_fc2.get_layer(layer_name).trainable = False

In [75]:
# Do not shuffle the data! You'll lose the label ordering
generator = image.ImageDataGenerator()
batches = generator.flow_from_directory(path + 'train', target_size=(224, 224), class_mode='categorical', shuffle=False, batch_size=batch_size)
val_batches = generator.flow_from_directory(path + 'valid', target_size=(224, 224), class_mode='categorical', shuffle=False, batch_size=batch_size)
test_batches = generator.flow_from_directory(path + 'test', target_size=(224, 224), class_mode='categorical', shuffle=False, batch_size=batch_size)
from keras.utils.np_utils import to_categorical
(val_classes, trn_classes, val_labels, trn_labels) = \
(val_batches.classes, batches.classes, to_categorical(val_batches.classes), to_categorical(batches.classes))

Found 20812 images belonging to 257 classes.
Found 3673 images belonging to 257 classes.
Found 6122 images belonging to 257 classes.


In [76]:
from keras.optimizers import Adam
model_fc2.compile(Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
model_fc2.fit(conv_feat, trn_labels, batch_size=batch_size, epochs=5, validation_data=(conv_val_feat, val_labels))

Train on 20812 samples, validate on 3673 samples
Epoch 1/5
20812/20812 [==============================] - 31s - loss: 108.4324 - acc: 0.5838 - val_loss: 36.9812 - val_acc: 0.7049
Epoch 2/5
20812/20812 [==============================] - 31s - loss: 17.0935 - acc: 0.8159 - val_loss: 7.7838 - val_acc: 0.7100
Epoch 3/5
20812/20812 [==============================] - 31s - loss: 4.4639 - acc: 0.8377 - val_loss: 3.6446 - val_acc: 0.7139
Epoch 4/5
20812/20812 [==============================] - 31s - loss: 2.6589 - acc: 0.8356 - val_loss: 2.8867 - val_acc: 0.7152
Epoch 5/5
20812/20812 [==============================] - 31s - loss: 2.3435 - acc: 0.8276 - val_loss: 2.7126 - val_acc: 0.7190


In [70]:
model_fc2.fit(conv_feat, trn_labels, batch_size=batch_size, epochs=4, validation_data=(conv_val_feat, val_labels))

Train on 20812 samples, validate on 3673 samples
Epoch 1/4
20812/20812 [==============================] - 31s - loss: 3.3993 - acc: 0.7616 - val_loss: 3.2209 - val_acc: 0.7062
Epoch 2/4
20812/20812 [==============================] - 31s - loss: 2.9087 - acc: 0.7456 - val_loss: 3.0185 - val_acc: 0.6986
Epoch 3/4
15296/20812 [=====================>........] - ETA: 7s - loss: 2.7969 - acc: 0.7510

KeyboardInterrupt: 

In [55]:
model_fc2.fit(conv_feat, trn_labels, batch_size=batch_size, epochs=4, validation_data=(conv_val_feat, val_labels))

Train on 20812 samples, validate on 3673 samples
Epoch 1/4
20812/20812 [==============================] - 23s - loss: 0.5555 - acc: 0.8581 - val_loss: 0.9640 - val_acc: 0.7642
Epoch 2/4
20812/20812 [==============================] - 23s - loss: 0.4709 - acc: 0.8802 - val_loss: 0.9563 - val_acc: 0.7699
Epoch 3/4
20812/20812 [==============================] - 23s - loss: 0.3984 - acc: 0.8981 - val_loss: 0.9489 - val_acc: 0.7672
Epoch 4/4
20812/20812 [==============================] - 23s - loss: 0.3443 - acc: 0.9121 - val_loss: 0.9475 - val_acc: 0.7716


In [56]:
model_fc2.fit(conv_feat, trn_labels, batch_size=batch_size, epochs=4, validation_data=(conv_val_feat, val_labels))

Train on 20812 samples, validate on 3673 samples
Epoch 1/4
20812/20812 [==============================] - 23s - loss: 0.2979 - acc: 0.9252 - val_loss: 0.9426 - val_acc: 0.7727
Epoch 2/4
20812/20812 [==============================] - 23s - loss: 0.2612 - acc: 0.9334 - val_loss: 0.9443 - val_acc: 0.7759
Epoch 3/4
20812/20812 [==============================] - 23s - loss: 0.2334 - acc: 0.9413 - val_loss: 0.9485 - val_acc: 0.7727
Epoch 4/4
20812/20812 [==============================] - 22s - loss: 0.1997 - acc: 0.9525 - val_loss: 0.9486 - val_acc: 0.7743


In [31]:
model_fc2.fit(conv_feat, trn_labels, batch_size=batch_size, epochs=4, validation_data=(conv_val_feat, val_labels))

Train on 20812 samples, validate on 3673 samples
Epoch 1/4
20812/20812 [==============================] - 21s - loss: 0.4855 - acc: 0.8855 - val_loss: 1.3324 - val_acc: 0.7457
Epoch 2/4
20812/20812 [==============================] - 21s - loss: 0.4494 - acc: 0.8936 - val_loss: 1.3485 - val_acc: 0.7367
Epoch 3/4
20812/20812 [==============================] - 21s - loss: 0.3998 - acc: 0.9065 - val_loss: 1.3356 - val_acc: 0.7416
Epoch 4/4
20812/20812 [==============================] - 21s - loss: 0.3683 - acc: 0.9118 - val_loss: 1.3877 - val_acc: 0.7408


In [80]:
%%time
j = 0
for i in range(1000):
    j += 1

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 161 µs
